In [1]:
import torch
import numpy as np
import torchvision
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from torchvision import datasets, transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn.functional as F

In [2]:
class Network(nn.Module):
    
    def __init__(self):

        super().__init__()

        # sees 32, 32, 3
        self.conv1 = nn.Conv2d(3, 64, 11, padding=5)
        self.batch_norm1 = nn.BatchNorm2d(64)
        
        # sees 16, 16, 64
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(128)
        
        # sees 16, 16, 128
        self.conv3 = nn.Conv2d(128, 128, 3, padding=1)
        self.batch_norm3 = nn.BatchNorm2d(128)
        
        self.batch_norm_mean_pool = nn.BatchNorm1d(128)
        
        # will get flattened to 128
        self.fc1 = nn.Linear(128, 10)
        
        
    
        # sees 16, 16, 128 -> will flatten to 128
        self.global_mean_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.max_pool = nn.MaxPool2d(2, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):

        x = self.max_pool(F.relu(self.batch_norm1(self.conv1(x))))
        
        x = F.relu(self.batch_norm2(self.conv2(x)))
        
        x = F.relu(self.batch_norm3(self.conv3(x)))

        # global mean pool
        x = self.global_mean_pool(x)
        
        # flatten (remove the 1 dimensions)
        x = torch.squeeze(x)
        
        x = self.batch_norm_mean_pool(x)
        
        x = F.relu(self.fc1(x))
        x = self.softmax(x)

        return x

In [3]:
# load CIFAR image data

In [4]:
BATCH_SIZE = 50

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# create network object
model = Network()

In [ ]:
# define the optimizer and loss function

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Device configuration
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cuda:0'

In [ ]:
# move to gpu
model.to(device)

Network(
  (conv1): Conv2d(3, 64, kernel_size=(11, 11), stride=(1, 1), padding=(5, 5))
  (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_norm3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm_mean_pool): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=128, out_features=10, bias=True)
  (global_mean_pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (softmax): Softmax()
)

In [ ]:
# train the model
epochs = 150
total_steps = len(trainloader)
training_losses = []
test_losses = []
train_acc = []
test_acc = []

# loop through epochs
for epoch in range(epochs):
    
    train_running_loss = 0  # track train running loss
    correct = 0
    total = 0
    
    model.train() # set to train mode
    
    # load batch images/labels
    for step, (images, labels) in enumerate(trainloader):
        
        # put data onto available device
        images = images.to(device)
        labels = labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
    
        outputs = model(images)  # forward pass
        _, predicted = torch.max(outputs.data, 1)  # retrieve top preds
        
        total += labels.size(0)  # add batch size
        correct += (predicted == labels).sum().item()  # calc num correct
        
        loss = criterion(outputs, labels)  # calc loss
        train_running_loss += loss.item()  # acc running loss
        
        loss.backward()   # backprop
        optimizer.step()  # forward

        if step % 100 == 0:  # print progress by iteration
        
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
            .format(epoch+1, epochs, step+1, total_steps, loss.item()))
    
    # div by num batches to get average
    epoch_train_loss = train_running_loss / len(trainloader)
                   
    print('Epoch [{}/{}], Train Loss: {:.4f}'.format(epoch+1, epochs, epoch_train_loss))
        
    # append the loss/acc after all the steps 
    training_losses.append(epoch_train_loss)
    train_acc.append(correct / total)
        
    
    # ------------------------------ #
    

    # evaluate on test data
    model.eval()
    with torch.no_grad():
        test_running_loss = 0  # track test running loss
        correct = 0
        total = 0

        for images, labels in testloader:
                   
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
                   
            loss = criterion(outputs, labels)
            test_running_loss += loss.item()
                   
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
                 
    # div by num batches
    epoch_test_loss = test_running_loss / len(testloader)
                   
    print('Epoch [{}/{}], Test Loss: {:.4f}'.format(epoch+1, epochs, epoch_test_loss))
        
    # append the loss & acc after all the steps 
    test_losses.append(epoch_test_loss)
    test_acc.append(correct / total)
            
    print('Test Accuracy: {} %'.format(100 * correct / total))

Epoch [1/150], Step [1/1000], Loss: 2.3015
Epoch [1/150], Step [101/1000], Loss: 2.2406
Epoch [1/150], Step [201/1000], Loss: 2.1414
Epoch [1/150], Step [301/1000], Loss: 2.1390
Epoch [1/150], Step [401/1000], Loss: 2.1205
Epoch [1/150], Step [501/1000], Loss: 2.1148
Epoch [1/150], Step [601/1000], Loss: 1.9734
Epoch [1/150], Step [701/1000], Loss: 2.0385
Epoch [1/150], Step [801/1000], Loss: 2.0093
Epoch [1/150], Step [901/1000], Loss: 2.0009
Epoch [1/150], Train Loss: 2.1009
Epoch [1/150], Test Loss: 2.0442
Test Accuracy: 41.59 %
Epoch [2/150], Step [1/1000], Loss: 2.0271
Epoch [2/150], Step [101/1000], Loss: 2.0539
Epoch [2/150], Step [201/1000], Loss: 1.9827
Epoch [2/150], Step [301/1000], Loss: 2.0336
Epoch [2/150], Step [401/1000], Loss: 1.8810
Epoch [2/150], Step [501/1000], Loss: 1.9870
Epoch [2/150], Step [601/1000], Loss: 1.8895
Epoch [2/150], Step [701/1000], Loss: 1.9674
Epoch [2/150], Step [801/1000], Loss: 2.0263
Epoch [2/150], Step [901/1000], Loss: 1.9079
Epoch [2/150],

Epoch [16/150], Step [101/1000], Loss: 1.7238
Epoch [16/150], Step [201/1000], Loss: 1.7147
Epoch [16/150], Step [301/1000], Loss: 1.6875
Epoch [16/150], Step [401/1000], Loss: 1.7892
Epoch [16/150], Step [501/1000], Loss: 1.7152
Epoch [16/150], Step [601/1000], Loss: 1.7385
Epoch [16/150], Step [701/1000], Loss: 1.7608
Epoch [16/150], Step [801/1000], Loss: 1.6967
Epoch [16/150], Step [901/1000], Loss: 1.7461
Epoch [16/150], Train Loss: 1.7180
Epoch [16/150], Test Loss: 1.7478
Test Accuracy: 71.66 %
Epoch [17/150], Step [1/1000], Loss: 1.7408
Epoch [17/150], Step [101/1000], Loss: 1.8273
Epoch [17/150], Step [201/1000], Loss: 1.6937
Epoch [17/150], Step [301/1000], Loss: 1.6339
Epoch [17/150], Step [401/1000], Loss: 1.7248
Epoch [17/150], Step [501/1000], Loss: 1.6646
Epoch [17/150], Step [601/1000], Loss: 1.6041
Epoch [17/150], Step [701/1000], Loss: 1.6937
Epoch [17/150], Step [801/1000], Loss: 1.7535
Epoch [17/150], Step [901/1000], Loss: 1.7431
Epoch [17/150], Train Loss: 1.7069
E

Test Accuracy: 76.28 %
Epoch [31/150], Step [1/1000], Loss: 1.6493
Epoch [31/150], Step [101/1000], Loss: 1.6104
Epoch [31/150], Step [201/1000], Loss: 1.6215
Epoch [31/150], Step [301/1000], Loss: 1.6768
Epoch [31/150], Step [401/1000], Loss: 1.5262
Epoch [31/150], Step [501/1000], Loss: 1.6055
Epoch [31/150], Step [601/1000], Loss: 1.6434
Epoch [31/150], Step [701/1000], Loss: 1.6556
Epoch [31/150], Step [801/1000], Loss: 1.5999
Epoch [31/150], Step [901/1000], Loss: 1.6202
Epoch [31/150], Train Loss: 1.6257
Epoch [31/150], Test Loss: 1.6973
Test Accuracy: 76.41 %
Epoch [32/150], Step [1/1000], Loss: 1.5708
Epoch [32/150], Step [101/1000], Loss: 1.5688
Epoch [32/150], Step [201/1000], Loss: 1.6535
Epoch [32/150], Step [301/1000], Loss: 1.5634
Epoch [32/150], Step [401/1000], Loss: 1.5378
Epoch [32/150], Step [501/1000], Loss: 1.6138
Epoch [32/150], Step [601/1000], Loss: 1.6068
Epoch [32/150], Step [701/1000], Loss: 1.7288
Epoch [32/150], Step [801/1000], Loss: 1.6527
Epoch [32/150],

Epoch [45/150], Train Loss: 1.5825
Epoch [45/150], Test Loss: 1.6827
Test Accuracy: 78.17 %
Epoch [46/150], Step [1/1000], Loss: 1.6140
Epoch [46/150], Step [101/1000], Loss: 1.5955
Epoch [46/150], Step [201/1000], Loss: 1.5614
Epoch [46/150], Step [301/1000], Loss: 1.5296
Epoch [46/150], Step [401/1000], Loss: 1.5231
Epoch [46/150], Step [501/1000], Loss: 1.6142
Epoch [46/150], Step [601/1000], Loss: 1.5748
Epoch [46/150], Step [701/1000], Loss: 1.5049
Epoch [46/150], Step [801/1000], Loss: 1.5592
Epoch [46/150], Step [901/1000], Loss: 1.5803
Epoch [46/150], Train Loss: 1.5799
Epoch [46/150], Test Loss: 1.6916
Test Accuracy: 77.14 %
Epoch [47/150], Step [1/1000], Loss: 1.5645
Epoch [47/150], Step [101/1000], Loss: 1.5789
Epoch [47/150], Step [201/1000], Loss: 1.4922
Epoch [47/150], Step [301/1000], Loss: 1.5733
Epoch [47/150], Step [401/1000], Loss: 1.5901
Epoch [47/150], Step [501/1000], Loss: 1.5382
Epoch [47/150], Step [601/1000], Loss: 1.5913
Epoch [47/150], Step [701/1000], Loss:

Epoch [60/150], Step [801/1000], Loss: 1.5064
Epoch [60/150], Step [901/1000], Loss: 1.4952
Epoch [60/150], Train Loss: 1.5488
Epoch [60/150], Test Loss: 1.6834
Test Accuracy: 77.75 %
Epoch [61/150], Step [1/1000], Loss: 1.5009
Epoch [61/150], Step [101/1000], Loss: 1.5619
Epoch [61/150], Step [201/1000], Loss: 1.5290
Epoch [61/150], Step [301/1000], Loss: 1.5062
Epoch [61/150], Step [401/1000], Loss: 1.5702
Epoch [61/150], Step [501/1000], Loss: 1.4835
Epoch [61/150], Step [601/1000], Loss: 1.5341
Epoch [61/150], Step [701/1000], Loss: 1.5745
Epoch [61/150], Step [801/1000], Loss: 1.5074
Epoch [61/150], Step [901/1000], Loss: 1.5750
Epoch [61/150], Train Loss: 1.5464
Epoch [61/150], Test Loss: 1.6812
Test Accuracy: 77.94 %
Epoch [62/150], Step [1/1000], Loss: 1.5298
Epoch [62/150], Step [101/1000], Loss: 1.5411
Epoch [62/150], Step [201/1000], Loss: 1.5821
Epoch [62/150], Step [301/1000], Loss: 1.6225
Epoch [62/150], Step [401/1000], Loss: 1.5379
Epoch [62/150], Step [501/1000], Loss:

Epoch [75/150], Step [601/1000], Loss: 1.5317
Epoch [75/150], Step [701/1000], Loss: 1.4789
Epoch [75/150], Step [801/1000], Loss: 1.5566
Epoch [75/150], Step [901/1000], Loss: 1.4920
Epoch [75/150], Train Loss: 1.5284
Epoch [75/150], Test Loss: 1.6708
Test Accuracy: 79.22 %
Epoch [76/150], Step [1/1000], Loss: 1.4749
Epoch [76/150], Step [101/1000], Loss: 1.5252
Epoch [76/150], Step [201/1000], Loss: 1.4919
Epoch [76/150], Step [301/1000], Loss: 1.5178
Epoch [76/150], Step [401/1000], Loss: 1.5320
Epoch [76/150], Step [501/1000], Loss: 1.5958
Epoch [76/150], Step [601/1000], Loss: 1.5394
Epoch [76/150], Step [701/1000], Loss: 1.4880
Epoch [76/150], Step [801/1000], Loss: 1.5699
Epoch [76/150], Step [901/1000], Loss: 1.4956
Epoch [76/150], Train Loss: 1.5263
Epoch [76/150], Test Loss: 1.6756
Test Accuracy: 78.38 %
Epoch [77/150], Step [1/1000], Loss: 1.4626
Epoch [77/150], Step [101/1000], Loss: 1.5384
Epoch [77/150], Step [201/1000], Loss: 1.5244
Epoch [77/150], Step [301/1000], Loss:

In [ ]:
def plot_graph(train_losses, test_losses, train_acc, test_acc):
    # plot graph
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label="Train loss")
    plt.plot(test_losses, label="Test loss")
    plt.legend(loc='best')
    plt.title("Loss vs Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Loss (Cross entropy)")

    plt.subplot(1, 2, 2)
    plt.plot(train_acc, label="Train Accuracy")
    plt.plot(test_acc, label="Test Accuracy")
    plt.legend(loc='best')
    plt.title("Accuracy vs Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.show()
    plt.savefig('part_a_cifar_loss_acc.png')

In [ ]:
plot_graph(training_losses, test_losses, train_acc, test_acc)

In [ ]:
model

In [ ]:
# retrieve conv1 layer, convert to numpy array, put on cpu first
weights = model.conv1.weight.data.cpu()

In [ ]:
def norm_stretch(filter):
    
    filter = filter - torch.min(filter)
    
    return filter / torch.max(filter)

In [ ]:
# plot filters
def plot_kernels(tensor, num_rows=8, num_cols=8):

    fig = plt.figure(figsize=(num_cols,num_rows))
    
    for i in range(tensor.shape[0]):
        ax1 = fig.add_subplot(num_rows,num_cols,i+1)  
        
        # need to reshape it to move channels to end
        reshaped = tensor[i].view(11,11,3)
        norm_tensor = norm_stretch(reshaped)
        
#         print(norm_tensor.shape)
        
#         ax1.imshow(tensor[i])
        ax1.imshow(norm_tensor)
        ax1.axis('off')
        ax1.set_xticklabels([])
        ax1.set_yticklabels([])
    
#     plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig('filters_part3b.jpg')
    plt.show()
    plt.clf()
    

In [ ]:
plot_kernels(weights)